# Terraform

- Install:
  - https://developer.hashicorp.com/terraform/install
- Language:
  - https://developer.hashicorp.com/terraform/language
  - https://developer.hashicorp.com/terraform/language/providers
- Providers
  - https://registry.terraform.io/browse/providers
  - https://registry.terraform.io/providers/hashicorp/azurerm/latest
  - https://registry.terraform.io/providers/hashicorp/kubernetes/latest

## Show Terraform version

In [1]:
!terraform --version

Terraform v1.7.2
on linux_amd64


## Show Terraform help

- The command `terraform --help` shows help about terraform commands.
- The command `terraform [Command] --help` shows additional help about a specific command, e.g. `terraform apply --help`.

In [2]:
!terraform --help

Usage: terraform [global options] <subcommand> [args]

The available commands for execution are listed below.
The primary workflow commands are given first, followed by
less common or more advanced commands.

Main commands:
  init          Prepare your working directory for other commands
  validate      Check whether the configuration is valid
  plan          Show changes required by the current configuration
  apply         Create or update infrastructure
  destroy       Destroy previously-created infrastructure

All other commands:
  console       Try Terraform expressions at an interactive command prompt
  fmt           Reformat your configuration in the standard style
  force-unlock  Release a stuck lock on the current workspace
  get           Install or upgrade remote Terraform modules
  graph         Generate a Graphviz graph of the steps in an operation
  import        Associate existing infrastructure with a Terraform resource
  login         Obtain and save credentials for a

## View  `providers.tf`

- This file configures Terraform providers.
- A provider can be seen as a plugin with support for defining resources for a specific provider.
- In this case, only one provier is configured:
  - The `azurerm` provider provides functionality for defining Azure resource.

In [3]:
# !type providers.tf # use this on Windows
!cat providers.tf

# Initialises Terraform providers and sets their version numbers.
# Note:
# 1) We require the provider "hashicorp/azurerm" version "3.90.0" (a specific Azure provider).
# 2) We require the version of Terrafor to be "1.7.2".
# 3) We are using the provider "azurerm" (with default features) that we required in (1).

terraform {
  required_providers {
    azurerm = {
      source  = "hashicorp/azurerm"
      version = "3.90.0"
    }
  }

  required_version = "= 1.7.2"
}

provider "azurerm" {
  features {}
}

## Initialize Terraform

- This will:
  - Download the configured provider `azurerm` (in `providers.tf`) to the folder `.terraform`.
  - Store version information about the provider `azurerm` in the file `.terraform.lock.hcl`.

In [4]:
#!rm -rf .terraform rm .terraform.lock.hcl terraform.tfstate terraform.tfstate.backup
!terraform init


Initializing the backend...

Initializing provider plugins...
- Finding hashicorp/azurerm versions matching "3.90.0"...
- Installing hashicorp/azurerm v3.90.0...
- Installed hashicorp/azurerm v3.90.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


## Terraform Fmt

- This command formats `.tf` files in the current folder (e.g. correct indentation, etc.).

In [5]:
!terraform fmt

## View `variables.tf`

- This file defines Terraform variables.
- A variable is defined as a name-value pair.
- In this case only two variables are defined.
  - The variable with name `app_name` and value `tsfn14g00`.
    - In this exercise, I'm using this variable to provide a name for an entire Azure application.
  - The variable with name `location` and value `westeurope`.
    - In this exercise, I'm using this variable to determine what Azure `zone/region` to use for storing my Azure resources.
      - A `zone/region` relates to one of Microsoft's data centers (in this case one that is located in Western Europe).

**Note**

- Change the value of the `app_name` variable from `tsfn14g00` to something unique.
  - This is required since it is used to defined URLs that have to be globally unique.
  - The pattern I have used is:
    - The course code `tsfn14`.
    - Followed by a `g` for `group`.
    - And finally a `number` representing a `group number` (in my case I am using group `00`).

In [6]:
#!type variables.tf # use this on Windows
!cat variables.tf

# Sets global variables for this Terraform project.
# Note:
# - The block type is "variable" since we are defining a Terraform variable.
# - "app_name" and "location" is the names of the Terraform variables.
# - "default" is used to set the value for a Terraform variable.
# - if "default" is omitted, Terraform will ask the user to input the value for the variable during "terraform apply".

variable "app_name" {
  default = "tsfn14g00"
}

variable "location" {
  default = "westeurope"
}

## View `resource-group.tf`

- This file defines an Azure `Resource Group`.
  - A `Resource Group` is used as a `namespace` in which various resources can be logically organized.
  - All resources contained in a `Resource Group` are destroyed when the `Resource Group` is destroyed.
- A resouce is defined using:
  - The keyword `resource`.
  - Followed by a quoted string containing:
    - The name of the provider (in this case `azurerm`).
    - Followed by an underscore (`-`).
    - Then the name of the provider's resource you want to use (in this case `resource_group` for an Azure Resource Group).
  - The next quoted string gives this resource a unique name, used to identify it in this and other Terraform files (in this case `main`).
  - Everything within brackets (`{ }`) then configures the resource with resource-specific name-value pairs.
    - An Azure Resource Group needs a unqiue `name` and `location`.
    - In this case, the variables defined in the file `variables.tf` are used to set these properties.
      - You use a variable's value by preceeding the variable's name with the word `var` (then a dot `.` and the variable's name).
- This file defines an Azure Resource Group
  - With the `name` set to `tsfn14g00` (according to the current value of the variable `app_name` in the file `variables.tf`).
  - And the `localtion` et to `westeurope` (according to the current value of the variable `location` in the file `variables.tf`).

In [7]:
#!type resource-group.tf #use this on Windows
!cat resource-group.tf

# Creates a resource group in your Azure account.
# Note:
# - The block type is "resource" since we are defining a resource.
# - The resource we are defining is "azurerm_resource_group" (an Azure Resource Group).
# - "main" is the name used to identify the resource in Terraform as "azurerm_resource_group.main"
# - "name" is the name of the resource group in Azure.
# - "location" is the region in Azure (where the resource group metadata will be stored).
# - Notice how the Terrform variables (defined in the file "variables.tf") are used:
#   - "var.app_name" refers to the variable "app_name" and returns its associated value.
#   - "var.location" refers to the variable "location" and returns its associated value.

resource "azurerm_resource_group" "main" {
  name     = var.app_name
  location = var.location
}

## Terraform Plan

- This will show Terraform's execution plan (without executing it) for all `.tf` files in the current directory.
  - All Terraform files (`.tf`) in the current directory will be merged in memory by the Terraform CLI.
  - Then Terraform calculates what changes need to be make to set up teh infrastructure (resources) defined in the files.
  - Finally, Terraform produces an `executation plan` for setting up the infrastructure (on Azure in this case).
    - The `executation plan` isn't executed at this time, it is only used to verify everything looks okey.

In [8]:
!terraform plan


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # azurerm_resource_group.main will be created
  + resource "azurerm_resource_group" "main" {
      + id       = (known after apply)
      + location = "westeurope"
      + name     = "tsfn14g00"
    }

Plan: 1 to add, 0 to change, 0 to destroy.

───────────────────────────────────────────────────────────────────────────────

Note: You didn't use the -out option to save this plan, so Terraform can't
guarantee to take exactly these actions if you run "terraform apply" now.


## Terraform Apply

- This will apply Terraform's execution plan for all `.tf` files in the current directory.
- The current Terraform state will be stored in the files `terraform.tfstate` and `.terraform.lock.hcl`.
  - Terraform uses these files to compare the current state of the infrastructure with the state in the Terraform files.
    - This is how Terraform knows how to update existing infrastructure.

### Note!

The command `terraform apply` can be used:
- Without `-auto-approve`
  -  Terraform will ask for a confirmation (`yes` or `no`) before applying.
- With `-auto-approve`
  - Terrafowm will **not** ask for any confirmation before applying.

In [9]:
#!terraform apply -auto-approve
!terraform apply -auto-approve


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # azurerm_resource_group.main will be created
  + resource "azurerm_resource_group" "main" {
      + id       = (known after apply)
      + location = "westeurope"
      + name     = "tsfn14g00"
    }

Plan: 1 to add, 0 to change, 0 to destroy.
azurerm_resource_group.main: Creating...
azurerm_resource_group.main: Creation complete after 1s [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00]

Apply complete! Resources: 1 added, 0 changed, 0 destroyed.


## List Azure Resource Groups

- The command below can be used to list all resoure groups on Azure.

In [10]:
!az group list -o table

Name       Location    Status
---------  ----------  ---------
tsfn14g00  westeurope  Succeeded


## List Terraform state

- This command will list all Terraform state (resources) contained in the file `terraform.state`.

In [11]:
!terraform state list

azurerm_resource_group.main


## Show Terraform state for `azurerm_resource_group.main`

- This will show the definition for the Terraform's state `azurerm_resource_group.main` in the file `terraform.state`.

In [12]:
!terraform state show azurerm_resource_group.main

# azurerm_resource_group.main:
resource "azurerm_resource_group" "main" {
    id       = "/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00"
    location = "westeurope"
    name     = "tsfn14g00"
}


## Show all Terraform state

- This will show the definition for all Terraform's state in the file `terraform.state`.

In [13]:
!terraform show

# azurerm_resource_group.main:
resource "azurerm_resource_group" "main" {
    id       = "/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00"
    location = "westeurope"
    name     = "tsfn14g00"
}


## Terraform Destroy

- This command will:
  - Destroy all resources (infrastructure) defined in the Terraform files-
  - Backup the old Terraform state to the file `terraform.tfstate.backup`.
  - Update the file `terraform.tfstate` with the new Terraform state.

### Note!

- The command can be executed:
  - Without `-auto-approve`
    -  Terraform will ask for a confirmation (`yes` or `no`) before destroying.
  - With `-auto-approve`
    - Terrafowm will **not** ask for any confirmation before destroying.

In [14]:
!terraform destroy -auto-approve

azurerm_resource_group.main: Refreshing state... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # azurerm_resource_group.main will be destroyed
  - resource "azurerm_resource_group" "main" {
      - id       = "/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00" -> null
      - location = "westeurope" -> null
      - name     = "tsfn14g00" -> null
      - tags     = {} -> null
    }

Plan: 0 to add, 0 to change, 1 to destroy.
azurerm_resource_group.main: Destroying... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00]
azurerm_resource_group.main: Still destroying... [id=/subscriptions/dc438970-aa32-41b3-8fe2-f587309a0208/resourceGroups/tsfn14g00, 10s elapsed]
azurerm_resource_group.main: Destructi

## List Azure Resource Groups

- After running `terraform destroy`, there shouldn't be any Resource Groups in Azure.

In [15]:
!az group list -o table

## Show all Terraform state

- This will show the definition for all Terraform's state in the file `terraform.state`.

In [16]:
!terraform show

The state file is empty. No resources are represented.
